# How to write functions

## Outline

* Guards in functions
* Helper construction `let` 
* Helper construction `where`
* Using both together

## Guards

Guards work similar like pattern matching. You can match a value against different cases and return the result for each case seperatly. The symbol `|` is used for guards. You also have the option to cover the case of all other posibilities, for which the keyword `otherwise` is used.

In [ ]:
testValue :: Int -> String
testValue n | n < 0 = "Smaller then 0."
            | n > 0 = "Larger then 0."
            | otherwise = "Equals 0."

main :: IO ()
main = do
    print "Type in a number:"
    n <- read <$> getLine
    print $ testValue n

## Let and where

We will learn how to make functions more convenient and readable using `let` and `where` constructions. We'll use the *multi-way if* construction to demonstrate them.

### Let

In the definition of the following function 

In [ ]:
calculateTime :: Int -> Int
calculateTime timeInS
        | timeInS < 40 = timeInS + 120
        | otherwise = timeInS + 8 + 120

some *magic numbers* (`40`, `120`, and `8`) are used. What are these numbers, what is their purpose, and is there a convenient way to refactor the function definition? Since none of these questions receive an answer, we say that these numbers are *magic*.       

In order to recover the true meaning of these values and avoid these problems we'll enter temporary expressions, and then the code becomes completely different:

In [ ]:
calculateTime :: Int -> Int
calculateTime timeInS =
    let threshold   = 40
        correction  = 120
        delta       = 8
    in
      if timeInS < threshold 
      then timeInS + correction
      else timeInS + delta + correction

We get rid of the *magic numbers* by introducing explanatory `threshold`, `correction`, and `delta` expressions through `let-in` construction.  

The `let-in` construction introduces clarifying expressions according to the scheme :

```haskell
let DECLARATIONS in EXPRESSION
```

where `DECLARATIONS` are the expressions we want to introduce and `EXPRESSION` is the expression, in which the expressions from `DECLARATION` are used.

When we write

```haskell
let threshold = 40
```

we declare: **"From now on, the expression `threshold` is equal to the expression 40."** It looks as an assignment, but we already know that there is no assignment in Haskell. Now the expression `threshold` can replace the number `40` inside the expression following the word `in`. 

**Benefits of `let-in` construction**

* Allows to introduce as many explanations for the following code as you like.

* Expressions in between `let` and `in` make our code clearer and in many cases even shorter.

**Warning about `let-in` construction:** the expression introduced by the `let-in` construction exists only within the expression following the word `in`.

The expression `delta` in the code below

In [ ]:
calculateTime :: Int -> Int
calculateTime timeInS =
    let threshold   = 40
        correction  = 120
    in
      if timeInS < threshold 
      then timeInS + correction
      else let delta = 8 
           in timeInS + delta + correction

is visible only inside the expression `timeInS + delta + correction`.

**Is it required to write every single expressions between `let` and `in` in separate line?** No, but they should be separated by the semicolon `;` as in the code below. 

In [ ]:
calculateTime :: Int -> Int
calculateTime timeInS =
    let threshold = 40; correction = 120
    in
      if timeInS < threshold 
      then timeInS + correction
      else let delta = 8 
           in timeInS + delta + correction

### Where

There is another way to introduce intermediate expressions using the `where` construction.

The `where` keyword does almost the same thing as `let-in`, but the intermediate expressions are set at the end of the function. It is similar to a style used in math and one should read it as a scientific formula:

```haskell
hundred = fifty * two
where
    fifty = 50
    two   = 2  
```

Using `where` the code for `calculateTime` function takes form

In [ ]:
calculateTime :: Int -> Int
calculateTime timeInS =
    if timeInS < threshold 
    then timeInS + correction
    else timeInS + delta + correction
    where
        threshold  = 40
        correction = 120
        delta      = 8

**How `where` differs from `let-in`?**

While `let-in` is used to create *super-local* expressions, the expressions in `where` are visible in any part of code preceding it.

### How to avoid writing the same code several times?

One of the ways to save yourself from writing a long formula several times and how you can maintain clean and readable code is to name a function inside `let-in` or `where` constructions, and use it in a code after `in` or before `where`. Therefore, we can treat complex functions even better. 

Let's work on example where we determine whether a given cylindrical shape is a glass, a bucket, or a tank depending on its volume. The parameters are diameter and height of a cylinder.

In [ ]:
analyzeCylinder :: Float -> Float -> String
analyzeCylinder diameter height = "The cylinder is " ++ 
    case volume of
        maxGlassVolume   -> "a glass."
        maxBucketVolume  -> "a bucket."
        maxTankVolume    -> "a tank."
        _                -> "something new to me..."
    where
        volume          = pi * diameter^2 * height / 4
        maxGlassVolume  = 100
        maxBucketVolume = 1000
        maxTankVolume   = 10000

### Let and where together

We can use `let-in` and `where` together, within the same function, but the general advise is: **do not mix up these constructions without any real nead**.

In the following function one part of expressions is located inside `let-in`, while the other part appers after `where` keyword.

In [ ]:
calculateTime :: Int -> Int
calculateTime timeInS =
    let threshold = 40 in
      if timeInS < threshold 
      then timeInS + correction
      else timeInS + delta + correction
    where
      correction = 120
      delta      = 8

**Can the expressions inside `let-in` and `where` depend on each other or on the parameter of the function?**

In all previous examples we've used only simple expressions, where the numbers were substituted by their names. However, both constructions allow much more complicated scenarious.

In [ ]:
calculateTime :: Int -> Int
calculateTime timeInS =
    let threshold = 40 in
      if timeInS < threshold 
      then timeInS + correction
      else timeInS + delta + correction
    where
      delta      = correction - 4
      correction = timeInS * 2

Now `delta` depends on `correction`, and `correction` depends on the parameter `timeInS`. 

**The order of appearance of expressions in `let-in` and `where` doesn't matter, even if one expression uses the other.**

In the following code `let`-expression uses the expression defined inside `where`:

In [ ]:
calculateTime :: Int -> Int
calculateTime timeInS =
    let delta     = correction - 4
        threshold = 40
    in
      if timeInS < threshold 
      then timeInS + correction
      else timeInS + delta + correction
    where
      correction = timeInS * 2

Here we've used the fact that `where`-expressions are visible in the any part of the code before `where`.

However, `let`-expressions aren't visible in `where`. The following code

In [ ]:
calculateTime :: Int -> Int
calculateTime timeInS =
    let delta     = correction - 4
        threshold = 40
    in
      if timeInS < threshold 
      then timeInS + correction
      else timeInS + delta + correction
    where
      correction = timeInS * 2 * threshold

returns an error 

        Not in scope: ‘threshold’

**Conclusion**: you cannot use `let`-expressions inside `where`-expressions, because the former are no longer included in the expression following the keyword `in`.

<div class="alert alert-block alert-warning">
<b>Warning!</b> Even if you can use let-expressions with where-expressions together, in most cases, you use one or the other.
</div>

## Summary

In this lesson we've discussed:

* Guards that work similiar as patter matching and are a cleaner way to write nested if-else statement when multiple conditions apply
* Let and where constructions allow to name constant parameters in code, to avoid using "magic numbers" and writing the same formulas several times, to make embedded espressions easier to read and they become "math recipies" for your formulas.